In [145]:
import os
import faiss
import json
import glob

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [146]:
os.environ["OpenAI_API_KEY"] = os.getenv("GPT_API")

model = ChatOpenAI(model ="gpt-4o")

In [147]:
# folder_path = "food_recipe/"

# # 병합된 데이터를 저장할 리스트
# merged_data = []

# # 폴더 내 모든 JSON 파일 읽기
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".json"):  # JSON 파일만 처리
#         file_path = os.path.join(folder_path, file_name)
#         with open(file_path, 'r', encoding='utf-8') as file:
#             try:
#                 # JSON 데이터 불러오기
#                 data = json.load(file)
#                 merged_data.append(data)  # 병합 리스트에 추가
#                 print(f"Added data from {file_name}")
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding {file_name}: {e}")

# # 병합된 데이터를 하나의 JSON 파일로 저장
# output_path = os.path.join(folder_path, "final_recipes_data.json")
# with open(output_path, 'w', encoding='utf-8') as output_file:
#     json.dump(merged_data, output_file, indent=4, ensure_ascii=False)

# print(f"병합된 JSON 데이터가 {output_path}에 저장되었습니다.")


In [148]:
# with open("food_recipe/food_recipes_1-68.json",'r',encoding='utf-8') as file:
#     data = json.load(file)

In [149]:
# document_recipe = [
#     Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
#     for recipe in data
# ]

In [150]:
# document_recipe

In [151]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [152]:
# index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

# recipe_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={}
# )

In [153]:
# uuids = [str(uuid4()) for _ in range(len(document_recipe))]

# recipe_store.add_documents(documents=document_recipe, ids = uuids)

In [154]:
# recipe_store.save_local("./food_db")
recipes = FAISS.load_local("./food_db." ,embeddings, allow_dangerous_deserialization=True)

In [155]:
retriever = recipes.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [156]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),
    ("system", "너는 요리의 전문가야."),
    
    ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),
    ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),
    ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),
    ("system", "질문에 대한 답변을 생성 하기전에 검증을 마친 후에 생성해줘."),
    ("system", "부족한 부분이 있다고 생각하면, 다른 데이터를 참조해줘"),

    
    ("system", "다음은 답변 형식의 예시야: \n"
               "user: 미역국을 만들고 싶어.\n"
               "ai: 미역국의 재료는 ~입니다.\n"
               "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"
               "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),
    
    ("system", "또 다른 예시를 들어줄게:\n"
               "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"
               "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),
    
    ("system", "추가 예시:\n"
               "user: 5인분의 레시피로 수정해줘.\n"
               "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"
               "ai: ~~~.\n"
               "이때도 순서를 전부 알려줘"),
    
    ("system", "추가 예시:\n"
               "user: 된장찌개 끓이는법을 알려줘.\n"
               "ai: 된장찌개는 재료에 따라 나뉩니다. 현재 ~ 한 된장찌개 레시피가 있습니다 어떤 된장찌개 레시피를원하십니까?:\n"
               ),
    
    ("user", "다음과 같은 데이터를 학습해:\n{data}"),
    ("user", "그리고 질문에 답해:\n{question}")
])

In [ ]:
contextual_prompt = prompt

In [ ]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        return {"data": inputs["data"], "question": inputs["question"]}
    
rag_chain_divide = {
    "data" : retriever,
    "question" : DebugPassThrough()

# contextual_prompt에 기능을 분리시키는 프롬프트 입력 (~라면 a를 반환, ~라면 b를 반환)
} | DebugPassThrough() | ContextToText()| contextual_prompt | model 


#위의 반환된 값에 따라서 레시피를 뽑을지, 음식을 추천 할 지에 대해서 나눔.
#그에 따라서  contextual_prompt의 변경이 필요함.
rag_chain_progress = {
    "data" : retriever,
    "question" : DebugPassThrough()
} | DebugPassThrough() | ContextToText()| contextual_prompt | model


In [ ]:
while True:
    print("-----------------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_divide.invoke(query)
    
    
    
    print("Question : ", query)
    print(response.content)

-----------------------------
Question :  된장찌개
된장찌개는 다양한 종류가 있습니다. 예를 들어, 바지락된장찌개, 꽃게된장찌개 등이 있습니다. 어떤 종류의 된장찌개 레시피를 원하시는지 말씀해주시면 상세한 정보를 제공해드리겠습니다.
-----------------------------
Question :  
네, 주어진 데이터를 학습했습니다. 질문해주시면 답변 드리겠습니다.
-----------------------------
Question :  된장찌개
된장찌개는 여러 종류가 있습니다. 현재 데이터에는 바지락된장찌개와 꽃게된장찌개 레시피가 있습니다. 어떤 종류의 된장찌개 레시피를 원하십니까?
-----------------------------
Question :  꽃게로
어떤 요리를 만드시고 싶으신가요? 꽃게로 만들 수 있는 요리는 다음과 같습니다:

1. 꽃게찜
2. 꽃게탕
3. 꽃게튀김
4. 꽃게된장찌개
5. 간장게장

어떤 요리의 레시피나 정보를 원하시는지 알려주시면 상세히 안내해 드리겠습니다.
-----------------------------
Question :  
물론입니다. 질문해 주세요.
-----------------------------
